<a href="https://colab.research.google.com/github/mohammadzav23-ux/Mohammad_DTSC3020_Fall2025/blob/main/AI_bot_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio pandas scikit-learn torch transformers --quiet


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline

class ContextManager:
    def __init__(self):
        self.history = []

    def add(self, user_input, intent):
        self.history.append({"text": user_input, "intent": intent})

    def clear(self):
        self.history = []

class IntentClassifier:
    def __init__(self):
        self.classifier = pipeline(
            "zero-shot-classification",
            model="facebook/bart-large-mnli"
        )

    def predict_intent(self, text, candidate_labels):
        result = self.classifier(text, candidate_labels)
        return result["labels"][0], result["scores"][0]

class EduBot:
    def __init__(self, faq_file="faq.csv"):
        self.data = pd.read_csv(faq_file, encoding="utf-8-sig")
        self.data.columns = self.data.columns.str.strip().str.lower()
        self.intents = [
            "assignment deadline",
            "grading and attendance policy",
            "exam schedule",
            "general course information"
        ]
        self.intent_model = IntentClassifier()
        self.context = ContextManager()
        self.sentiment_model = pipeline("sentiment-analysis",
                                        model="distilbert-base-uncased-finetuned-sst-2-english")
        self.vectorizer = TfidfVectorizer()
        self.question_vectors = self.vectorizer.fit_transform(self.data["question"])

    def get_response(self, user_input):
        intent_label, zero_shot_conf = self.intent_model.predict_intent(user_input, self.intents)
        sims = cosine_similarity(self.vectorizer.transform([user_input]), self.question_vectors)
        best_idx = sims.argmax()
        tfidf_sim = sims[0][best_idx]
        hybrid_confidence = 0.5 * zero_shot_conf + 0.5 * tfidf_sim
        answer = self.data.iloc[best_idx]["answer"]
        detected_intent = self.data.iloc[best_idx]["intent"]
        sentiment_result = self.sentiment_model(user_input)[0]
        sentiment = sentiment_result['label']
        sentiment_score = sentiment_result['score']
        if sentiment.lower() == "negative" and sentiment_score > 0.85 and "?" not in user_input:
            answer = "I understand your concern. " + answer
        self.context.add(user_input, detected_intent)
        return answer, detected_intent, hybrid_confidence, sentiment, sentiment_score


In [3]:
from google.colab import files
uploaded = files.upload()  # Select your 'faq.csv' file


Saving data:faq.csv to data:faq.csv


In [12]:
import gradio as gr
from datetime import datetime
import time
import threading

# Initialize bot
bot = EduBot("faq.csv")
chat_history = []

def chat(user_input, history):
    if not user_input.strip():
        return history, ""

    # Record user message
    timestamp_user = datetime.now().strftime("%I:%M %p")
    history.append({"sender": "user", "text": user_input, "time": timestamp_user})

    # Simulate typing indicator
    history.append({"sender": "bot", "text": "Typing...", "time": ""})

    # Function to generate bot response after delay
    def bot_response():
        response, intent, confidence, sentiment, sentiment_score = bot.get_response(user_input)

        # Emoji based on sentiment
        if sentiment.upper() == "NEGATIVE":
            response = "😟 " + response
        elif sentiment.upper() == "POSITIVE":
            response = "😃 " + response

        timestamp_bot = datetime.now().strftime("%I:%M %p")

        # Replace typing with actual response
        history[-1] = {"sender": "bot", "text": response, "time": timestamp_bot}

    t = threading.Thread(target=bot_response)
    t.start()
    t.join()  # Wait for bot to finish (simulate real-time typing)

    return history, ""  # Clear input box after sending

# Custom CSS for chat bubbles
css = """
<style>
.user_msg {
    background-color: #DCF8C6;
    color: black;
    padding: 10px;
    border-radius: 15px;
    margin: 5px;
    max-width: 70%;
    align-self: flex-end;
    word-wrap: break-word;
}
.bot_msg {
    background-color: #E5E5EA;
    color: black;
    padding: 10px;
    border-radius: 15px;
    margin: 5px;
    max-width: 70%;
    align-self: flex-start;
    word-wrap: break-word;
}
.time_stamp {
    font-size: 10px;
    color: gray;
    margin-top: 2px;
}
.chat_container {
    display: flex;
    flex-direction: column;
}
</style>
"""

# Function to format history into HTML chat bubbles
def format_chat(history):
    chat_html = "<div style='display:flex; flex-direction:column;'>"
    for msg in history:
        if msg["sender"] == "user":
            chat_html += f"""
            <div style='background-color:#DCF8C6; color:black; padding:10px; border-radius:15px; margin:5px; max-width:70%; align-self:flex-end; word-wrap:break-word;'>
                {msg['text']}
                <div style='font-size:10px; color:gray; margin-top:2px;'>{msg['time']}</div>
            </div>
            """
        else:  # bot message
            chat_html += f"""
            <div style='background-color:#E5E5EA; color:black; padding:10px; border-radius:15px; margin:5px; max-width:70%; align-self:flex-start; word-wrap:break-word;'>
                {msg['text']}
                <div style='font-size:10px; color:gray; margin-top:2px;'>{msg['time']}</div>
            </div>
            """
    chat_html += "</div>"
    return chat_html


Device set to use cpu
Device set to use cpu


In [14]:
with gr.Blocks() as demo:
    gr.Markdown("## 🎓 EduBot - AI Powered Chatbot")

    chatbot = gr.HTML(format_chat(chat_history), elem_id="chat_box")
    user_input = gr.Textbox(label="Type your message...", placeholder="Ask me anything...")
    send_btn = gr.Button("Send")

    def submit(user_input, history):
        history, cleared_input = chat(user_input, history)
        chat_html = format_chat(history)
        return chat_html, cleared_input

    send_btn.click(submit, inputs=[user_input, gr.State(chat_history)], outputs=[chatbot, user_input])
    user_input.submit(submit, inputs=[user_input, gr.State(chat_history)], outputs=[chatbot, user_input])

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://688fd9543465da8b28.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
